# Goal: creating a mini-site using Streamlit/Dash, where you can write down your ticker and all the data will be mined automatically
+ add all of this to Github and connect Git for it to be updated automatically (and for CV??)
+ add cash-flow/balance data with plots
+ add y/y growth to every cash-flow/balance/income
+ add average dividend yield and stuff and compare it to sector (seekingalpha.com)
+ add forecasts to plots (ARIMA or smth like that) + forecasts from analytics (seekingalpha.com)
+ add portfolio analysis (at least from etoro): sectors + dividend and price forecasting
+ add explanation to different ratios and indicators
+ add ETF analytics - sectors, top holdings (stockanalysis.com), overall position of different firms in portfolio
+ add different stuff based on alphaspread and seekingalpha examples
+ add news aggregator (maybe play a little bit with NLTK for text recognition and classification)
+ add grades from analytics and firms (current and historical, checked using logistic regression??)
+ add Monte Carlo simulation based on historical performance
+ add somewhere ML / Gradient Boosting / Decision Tree / etc. ??
+ add all of this to AWS (or other Cloud) for data to be updated automatically?

In [2]:
import yahoofinancials
import yahooquery as yq
import yfinance as yf
import numpy as np # this is for working with numbers
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import datetime as dt
import matplotlib.pyplot as plt
import requests
import json
import nasdaqdatalink as ndl
import time
import re
from bs4 import BeautifulSoup
import lxml
import cchardet

NASDAQ_DATA_LINK_API_KEY = "xy8jtvPFDhiwnFktEugz" # ndl.ApiConfig.api_key 
pd.set_option('display.max_columns', None)

In [5]:
STOCK = 'UNP'

STOCKS_LIST = ['AAPL', 'ABBV', 'ABR', 'ABT', 'AFL', 'AMD', 'AMGN', 'AMZN', 
    'ASML', 'ATVI', 'AVGO', 'BBY', 'BHP', 'BRK-B', 'CCI',
    'COST', 'CSCO', 'CVX', 'GOOGL', 'HD', 'IRM', 'JNJ', 'JPM',
    'LOW', 'MA', 'MAIN', 'MCD', 'MCO', 'MDT', 'META', 'MRK',
    'MS', 'MSFT', 'NFLX', 'NVDA', 'O', 'PEP', 'PFE', 'PLD', 'PYPL',
    'SBUX', 'STOR', 'TGT', 'TROW', 'TSLA',
    'TSM', 'UNP', 'UPS', 'V', 'VICI', 'WPC', 'XOM', 'ZTS']

CRYPTO_LIST = ['BTC-USD', 'ETH-USD']

ETF_LIST = ['SCHD', 'SPHD', 'VOO', 'QQQ', 'VGT', 'ARKK']

WATCHLIST_LIST = ['MO', 'T']

In [89]:
stock_list_new = [x.lower() if x != 'BRK-B' else 'brk.b' for x in STOCKS_LIST]
stock_list_test = [STOCK]

# API

SeekingAlpha dividend history

In [90]:
div_history = pd.DataFrame()
# seekingalpha dividend_history
for tick in stock_list_test:
    url = f"https://seekingalpha.com/api/v3/symbols/{tick.lower()}/dividend_history"
    querystring = {"years":"100"}
    headers = {
        "cookie": "machine_cookie=4979826528810; _cls_v=072cd8fc-83ec-4b6d-b840-72ce92a351d4; _cls_s=da78f999-6e82-4412-bfd3-98a35379d96d:0; _pxvid=6190f403-0540-11ed-8356-71796f6e5767; pxcts=61910480-0540-11ed-8356-71796f6e5767; g_state=^{^\^i_l^^:0^}; has_paid_subscription=false; OptanonAlertBoxClosed=2022-07-16T19:49:37.138Z; _ga=GA1.2.422884809.1658000977; _igt=80f0662b-29d6-4ba2-daef-f15a084be986; _hjSessionUser_65666=eyJpZCI6IjVmNjA3NTU1LTFmODItNWFhOC05NzBkLTMxNmIwOTFkNDJjZSIsImNyZWF0ZWQiOjE2NTgwNDMwMjQxNTYsImV4aXN0aW5nIjp0cnVlfQ==; _hjCachedUserAttributes=eyJhdHRyaWJ1dGVzIjp7ImxvZ2dlZF9pbiI6dHJ1ZSwibXBfc3ViIjpmYWxzZSwicHJlbWl1bV9zdWIiOmZhbHNlLCJwcm9fc3ViIjpmYWxzZX0sInVzZXJJZCI6IjU2ODczOTA0In0=; ga_clientid=422884809.1658000977; _pcid=^%^7B^%^22browserId^%^22^%^3A^%^22l6l1zvh16ggo2rl5^%^22^%^7D; _clck=1sv21qj^|1^|f4c^|0; _ig=56873904; sailthru_content=2528dc295dc3fbbf1ec8e71fd6af16ea5ed0fab1751712d30b586234ac21ac69c6f48017810681510ac670347a1b237b395addcc8a084ec17e397065464a467803e85c27969d6ca11adf1e5bae9ce43e365ade53ba1716e0f5409199ca81b1b2d336ff2bdab2770099e746360c3b2e4a8f46c8cbd3b263891ad28c66986af90e8a2bb0fb3446957f12521164830063aa9eada221935b05aaed9d45ccc5957509; sailthru_visitor=4a85db3b-194e-42bd-bc87-31076f836304; sailthru_hid=29f91ce2c0119534955a4934eea65d5d62d3164919e4cd8e5507453023d2712d74fca4d95585b51117583622; _gcl_au=1.1.905016176.1671643238; __pat=-18000000; user_id=56873904; user_nick=; user_devices=2; u_voc=; marketplace_author_slugs=; user_cookie_key=cjjdiz; user_perm=; sapu=101; user_remember_token=04b7dcb2602e3f78db1c7c7b3e0e43599aa202f5; _sapi_session_id=0pCP6BL7ckaTjzz1yGfnvj2fYymMCVyRcdc0FilJJuJrLs^%^2BPk6M7pmkTNZq^%^2Bs0tQzLw0Gwxfpuz4XXdeLwjnEvGdwVGKVQdIhiI4kf6GgA6c6Aqo8EAHDVX3JUirUkOfv7^%^2Fv6zuUolHyz^%^2Bka3l7tx2Tmr6LfeaHe0syKkJJ99iSM^%^2FbcPrEEdST3wciFuUBwzxt3V9trL98gAlWdoY4Ces0hsdCU^%^2BEryApHpHc9rt8S2ZjmXsQ7PNxkHufEwIxhqC2LmTKsoVyrOgYz4rWUiq8CGM^%^2BdxILxHnEzl1LN9h2hU^%^3D--^%^2Fq^%^2FbqzYaui40jz7x--I^%^2FfbuLyN7DqYI^%^2BHocBaR9A^%^3D^%^3D; _pctx=^%^7Bu^%^7DN4IgrgzgpgThIC5QFYBsAOA7AZgJwAYAWRUABxigDMBLAD0RBABoQAXAT1KgYDUANEAF9BLSLADKrAIatIDCgHNqEVrCgATZiAjVVASU0IAdmAA2pwUA; _pxhd=9b81b7053d831d0e418b92698dce0fc88c8297e1e67eb88e98fefc26b9d3b6ac:80650f60-6b3b-11e9-814e-41aaaa844f02; ubvt=b26b3487-0e8c-451d-9656-705df157b6a2; session_id=27a89810-0094-4454-8793-f52f76340fbd; OptanonConsent=isIABGlobal=false&datestamp=Thu+Dec+22+2022+16^%^3A05^%^3A26+GMT^%^2B0100+(czas+^%^C5^%^9Brodkowoeuropejski+standardowy)&version=6.30.0&landingPath=NotLandingPage&groups=C0001^%^3A1^%^2CC0002^%^3A1^%^2CC0003^%^3A1^%^2CC0007^%^3A1&hosts=H40^%^3A1^%^2CH17^%^3A1^%^2CH13^%^3A1^%^2CH36^%^3A1^%^2CH55^%^3A1^%^2CH69^%^3A1^%^2CH45^%^3A1^%^2CH14^%^3A1^%^2CH15^%^3A1^%^2CH19^%^3A1^%^2CH47^%^3A1&AwaitingReconsent=false&genVendors=V12^%^3A1^%^2CV5^%^3A1^%^2CV7^%^3A1^%^2CV8^%^3A1^%^2CV13^%^3A1^%^2CV15^%^3A1^%^2CV3^%^3A1^%^2CV2^%^3A1^%^2CV6^%^3A1^%^2CV14^%^3A1^%^2CV1^%^3A1^%^2CV4^%^3A1^%^2CV9^%^3A1^%^2C&geolocation=PL^%^3B14; __pnahc=1; gk_user_access=1**1671790151; gk_user_access_sign=316999477f1cf3b270ec2daee33355ef077c23cf; __tac=; __tae=1671790157992; LAST_VISITED_PAGE=^%^7B^%^22pathname^%^22^%^3A^%^22https^%^3A^%^2F^%^2Fseekingalpha.com^%^2Fsymbol^%^2FDPZ^%^2Fdividends^%^2Fhistory^%^22^%^2C^%^22pageKey^%^22^%^3A^%^22ba85820c-c9a7-4301-91ed-047be2dec0c2^%^22^%^7D; _uetsid=c9555410815311ed8383e1bd89176270; _uetvid=6c9a7a40054011ed9912e34a5318d584; __pvi=eyJpZCI6InYtMjAyMi0xMi0yMy0xMS0wOS0xNC0zMDYtRFVlQXM1NWtGcHdFelhldy05OWVlM2VhYmJkMDU0N2NiMjRiMjQ2ZTU5ZTc4YmQ4OCIsImRvbWFpbiI6Ii5zZWVraW5nYWxwaGEuY29tIiwidGltZSI6MTY3MTc5MDY2ODc4NH0^%^3D; __tbc=^%^7Bkpex^%^7Dc34b4dUSkelinBilgVjlXAFjdExL2yDTVVsaH2tHeWieSgu52a503DdkAZX5En4R; xbc=^%^7Bkpex^%^7DpsZvcg-czvsWNhuvqvMZK8J5UpYhUPaAf31G9LNO4s_JNybiiLibHlVRHn3hm4E4nn-OgFei0KNGMmPkAUA1_w-h83kuroSVs6Wm4u7Ywo2khMWDgt1X4fFsw_eRSpv_RT073ml6wbguc-BKt5xBC3jze6MTqMhOTtHPaQlo8jgrWISTUeJdpSW5wg1k8whSzoS5_JJNFGD12hP_7LIJ9Rcboio5C_pfp4SlYIgOvl0t0F4JUlwH3AItmjnB36P2lQd46Wi4gj8SrJp-WVo44vskLuAbTmezh-9Nmb6v2dAtnefy1d_SnhK1ucoCCPyx9eHnXkzHTxLTKoa4V1CaJBGXBFnLuyNvM48L074T6SRARQTZyVNljtYreNy7Uxb-agK4V0R54vP3iIc0NEPleFizxGh8FZZoF4flQb7mGezf-1HBFpWUlIR7p55GktmivP2SWPpXI1SzKXApvhhYN_mlYAm6eHG7Pq1LZgIR4zWUkv2RKy3rJd9Qsk8cHLPlvjhuRmx_t1ZjQa7IsxW7_03FS_lF67VC3PfVw_sI7vJlVj9ccU7hT9ptOtwx7ECKKYPkv5zP7q_a3Yubi4CmIM5MP-cJhy_-6RU96KhQ-FqXxVYETn_nJbtT3MXgwQma1soxbODUZ0d9NKNDWU5_lu9l2WXp88Vf-PdLt9LNv-Q",
        "authority": "seekingalpha.com",
        "referer": f"https://seekingalpha.com/symbol/{tick}/dividends/history",
        "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36"
    }
    response = requests.request("GET", url, headers=headers, params=querystring).json()['data']

    for id in response:
        row = pd.DataFrame([id['attributes']])
        row['Ticker'] = tick
        div_history = pd.concat([div_history, row], axis=0)
    # time.sleep(2) # in case of generating data on more then 1 ticker
div_history = div_history.reset_index(drop=True)
div_history

,year,amount,ex_date,freq,declare_date,pay_date,record_date,date,adjusted_amount,split_adj_factor,Ticker
0,1989,0.55,1989-06-05,QUARTERLY,None,1989-07-03,1989-06-09,1989-06-05,0.06875,8.0,UNP
1,1989,0.55,1989-09-07,QUARTERLY,None,1989-10-02,1989-09-13,1989-09-07,0.06875,8.0,UNP
2,1989,0.58,1989-12-07,QUARTERLY,None,1990-01-02,1989-12-13,1989-12-07,0.07250,8.0,UNP
3,1990,0.58,1990-03-01,QUARTERLY,None,1990-04-02,1990-03-07,1990-03-01,0.07250,8.0,UNP
4,1990,0.58,1990-06-07,QUARTERLY,None,1990-07-02,1990-06-13,1990-06-07,0.07250,8.0,UNP
...,...,...,...,...,...,...,...,...,...,...,...
130,2021,1.18,2021-12-17,QUARTERLY,2021-12-10,2021-12-30,2021-12-20,2021-12-17,1.18000,1.0,UNP
131,2022,1.18,2022-02-25,QUARTERLY,2022-02-03,2022-03-31,2022-02-28,2022-02-25,1.18000,1.0,UNP
132,2022,1.3,2022-05-27,QUARTERLY,2022-05-12,2022-06-30,2022-05-31,2022-05-27,1.30000,1.0,UNP
133,2022,1.3,2022-08-30,QUARTERLY,2022-07-28,2022-09-30,2022-08-31,2022-08-30,1.30000,1.0,UNP


NASDAQ dividend history

In [91]:
# url = "https://api.nasdaq.com/api/quote/AAPL/dividends"
# querystring = {"assetclass":"stocks"}

# headers = {
#     "authority": "api.nasdaq.com",
#     "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36"
# }
# response = requests.request("GET", url, headers=headers, params=querystring).json()

# ex_div_date = response['data']['exDividendDate']
# div_pay_date = response['data']['dividendPaymentDate']
# div_yield = response['data']['yield']
# annualized_div = response['data']['annualizedDividend']
# dividend_history = pd.DataFrame(response['data']['dividends']['rows'])
# dividend_history

StockAnalysis Financials

In [92]:
period = 'quarterly'
income_statements = pd.DataFrame()

for x in stock_list_test: # STOCKS_LIST
    print(x)

    url = f"https://stockanalysis.com/stocks/{x.lower()}/financials/__data.json"

    querystring = {"x-sveltekit-invalidated":"__1"} # also period can be "trailing" or no period at all (annual)
    if (period == 'quarterly')|(period == 'trailing'):
        querystring['period'] = period

    headers = {
        "authority": "stockanalysis.com",
        "accept": "*/*",
        "accept-language": "en-US,en;q=0.9,ru-RU;q=0.8,ru;q=0.7,uk-UA;q=0.6,uk;q=0.5,pl;q=0.4",
        "cookie": "cf_chl_2=9c80dd02f1fc73c; cf_clearance=OwRTeLsjteKSq2vOGA415U77v0RksWzpM_0xtiixnIA-1671821196-0-160",
        "referer": f"https://stockanalysis.com/stocks/{x.lower()}/financials/",
        "sec-ch-ua": "^\^Not?A_Brand^^;v=^\^8^^, ^\^Chromium^^;v=^\^108^^, ^\^Google",
        "sec-ch-ua-mobile": "?0",
        "sec-ch-ua-platform": "^\^Windows^^",
        "sec-fetch-dest": "empty",
        "sec-fetch-mode": "cors",
        "sec-fetch-site": "same-origin",
        "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36"
    }
    
    response = requests.request("GET", url, headers=headers, params=querystring).json()

    # with open("stock_analysis.json", "w") as outfile:
    #     json.dump(response, outfile)

    response_new = response['nodes']
    for i in response_new:
        if i['type'] != 'skip':
            data = list(i['data'])
    
    dict1 = {}
    for k, v in zip(data[0].keys(), data[0].values()):
        dict1[k] = data[v]

    dict2 = {}
    for k, v in zip(dict1['financialData'].keys(), dict1['financialData'].values()):
        dict2[k] = data[v]

    dict3 = {}
    for k, v_list in zip(dict2.keys(), dict2.values()):
        values = []
        for v in v_list:
            values = values + [data[v]]
        dict3[k] = values

    dict_names = {}
    for m in dict1['map']:
        dict_names[data[data[m]['id']]] = data[data[m]['title']]
    dict_names['fcf'] = 'Free Cash Flow'
    dict_names['datekey'] = 'Date'

    income_df = pd.DataFrame(dict3)
    income_df['Ticker'] = x
    income_df = income_df.rename(columns=dict_names)
    
    income_statements = pd.concat([income_statements, income_df], axis=0)

    # time.sleep(4) # in case of generating data on more then 1 ticker

income_statements = income_statements.set_index('Date')
income_statements['Dividends'] = income_statements['Dividend Per Share'] * income_statements['Shares Outstanding (Basic)']
income_statements.head(5)

UNP


,Revenue,Revenue Growth (YoY),Cost of Revenue,Gross Profit,"Selling, General & Admin",Research & Development,Operating Expenses,Other Operating Expenses,Operating Income,Interest Expense / Income,Other Expense / Income,Pretax Income,Income Tax,Preferred Dividends,Net Income,Net Income Growth,Shares Outstanding (Basic),Shares Outstanding (Diluted),Shares Change,EPS (Basic),EPS (Diluted),EPS Growth,Free Cash Flow Per Share,Dividend Per Share,Dividend Growth,Free Cash Flow,Gross Margin,Operating Margin,Profit Margin,Free Cash Flow Margin,Effective Tax Rate,EBITDA,EBITDA Margin,Depreciation & Amortization,EBIT,EBIT Margin,Ticker,Dividends
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2022-09-30,6566000000,0.179662,1773000000,4793000000,1278000000,0,2160000000,882000000,2633000000,315000000,-124000000,2442000000,547000000,0,1895000000,0.132696,620400000,621500000,-0.044287,3.05,3.05,0.186770,2.995,1.30,0.214953,1858000000,0.729973,0.401005,0.288608,0.282973,0.223997,3320000000,0.505635,563000000,2757000000,0.419890,UNP,806520000.0
2022-06-30,6269000000,0.138990,1792000000,4477000000,1092000000,0,1982000000,890000000,2495000000,316000000,-163000000,2342000000,507000000,0,1835000000,0.020578,625600000,626800000,-0.050447,2.93,2.93,0.077206,1.813,1.30,0.214953,1134000000,0.714149,0.397990,0.292710,0.180890,0.216482,3217000000,0.513160,559000000,2658000000,0.423991,UNP,813280000.0
2022-03-31,5860000000,0.171766,1490000000,4370000000,1101000000,0,1993000000,892000000,2377000000,307000000,-47000000,2117000000,487000000,0,1630000000,0.215511,632200000,633600000,-0.053198,2.58,2.57,0.285000,2.196,1.18,0.216495,1388000000,0.745734,0.405631,0.278157,0.236860,0.230043,2979000000,0.508362,555000000,2424000000,0.413652,UNP,745996000.0
2021-12-31,5733000000,0.115153,1365000000,4368000000,1070000000,0,1928000000,858000000,2440000000,295000000,-83000000,2228000000,517000000,0,1711000000,0.239855,653800000,655400000,-0.034899,2.66,2.66,0.297561,2.352,1.18,0.216495,1538000000,0.761905,0.425606,0.298448,0.268271,0.232047,3079000000,0.537066,556000000,2523000000,0.440084,UNP,771484000.0
2021-09-30,5566000000,0.131531,1271000000,4295000000,1040000000,0,1863000000,823000000,2432000000,290000000,-38000000,2180000000,507000000,0,1673000000,0.227439,648700000,650300000,-0.039155,2.58,2.57,0.278607,2.357,1.07,0.103093,1529000000,0.771649,0.436939,0.300575,0.274704,0.232569,3023000000,0.543119,553000000,2470000000,0.443766,UNP,694109000.0


In [93]:
# expenses dataframe for plot
expenses = [
    'Selling & Marketing',
    'General & Administrative',
    'Selling, General & Admin',
    'Research & Development',
    'Other Operating Expenses',
    "Interest Expense",
    'Interest Expense / Income',
    'Other Expense / Income'
    ]
df_expenses = income_statements.loc[:,[c for c in income_statements.columns if c in expenses]]
for c in df_expenses.columns:
    if 'Expense / Income' in c:
        for i in df_expenses.index:
            if df_expenses.loc[i,c]>0:
                df_expenses.loc[i,c] = 0
            else:
                df_expenses.loc[i,c] = -df_expenses.loc[i,c]
df_expenses.head(5)

,"Selling, General & Admin",Research & Development,Other Operating Expenses,Interest Expense / Income,Other Expense / Income
Date,,,,,
2022-09-30,1278000000,0,882000000,0,124000000
2022-06-30,1092000000,0,890000000,0,163000000
2022-03-31,1101000000,0,892000000,0,47000000
2021-12-31,1070000000,0,858000000,0,83000000
2021-09-30,1040000000,0,823000000,0,38000000


In [94]:
def create_plot_bar_line(bar='', line='', y2perc=False, secondary_y=True, bar_color='#5a7d9f', line_color='white'):
    fig = make_subplots(specs=[[{"secondary_y": True}]])

    fig.add_bar(
        x=income_statements.index,
        y=income_statements[bar], 
        name=bar, 
        secondary_y=False, 
        marker_color=bar_color
    )

    fig.update_layout(
        height=400, 
        width=600, 
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y = -0.15,
            xanchor="right",
            x = 0.75
        ),
        margin=dict(l=20, r=20, t=30, b=20),
        template="plotly_dark",
        title=bar
    )

    if line != '':
        fig.add_scatter(
            x=income_statements.index,
            y=income_statements[line], 
            name=line, 
            secondary_y=secondary_y, 
            marker_color=line_color
        )
        if y2perc:
            fig.update_layout(yaxis2=dict(tickformat='.0%'))

    fig.show()
    
    return fig

In [95]:
def create_line_plot(y:list, title='', perc=True):
    fig = go.Figure()
    
    for line in y:
        fig.add_scatter(
            x=income_statements.index, 
            y=income_statements[line], 
            name=line
        )

    fig.update_layout(
        height=400, 
        width=600, 
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y = -0.25,
            xanchor="right",
            x = 0.75
        ),
        margin=dict(l=20, r=20, t=30, b=20),
        template="plotly_dark",
        title=title
    )

    if perc:
        fig.update_layout(yaxis=dict(tickformat='.1%'))

    fig.show()
    return fig

In [96]:
def create_ema_plot(tickers_list, start_date='2019-01-01', period='days', emas=[1]):

    colors={'standard':'#424242', 
        'min':'#00ab41', 
        'other':'#8B8000', 
        'max':'#c30010'}
    
    df_orig = pd.DataFrame(yf.download(tickers_list, start_date)['Adj Close'])

    if len(tickers_list)>1:
        vertical_spacing = (1 / (len(tickers_list) - 1))/9
    else:
        vertical_spacing=0
        df_orig.columns = tickers_list

    df = df_orig.copy()
    
    for n in emas:
        ema_period = f"{n} {period}"
        ema_df = df_orig.ewm(halflife=ema_period, times=df_orig.index, ignore_na=False).mean()
        ema_df.columns = [f"{c}_{n}{period}_ema" for c in ema_df.columns]
        df = pd.concat([df, ema_df], axis=1)

    fig = make_subplots(
        rows=len(tickers_list),
        cols=1,
        subplot_titles=tickers_list,
        vertical_spacing=vertical_spacing)
    
    for i, c in enumerate(tickers_list, start=1):
        df_columns = [col for col in df.columns if col.startswith(c+'_')|(col==c)]
        df_new = df[df_columns].copy()
        for i2, c2 in enumerate(df_columns):

            if i2 == 0:
                color = colors['standard']
                width = 2
            elif i2 == 1:
                color = colors['min']
                width = 2
            elif i2 == len(df_columns)-1:
                color = colors['max']
                width = 2
            else:
                color = colors['other']
                width = 0.5
            
            fig.add_trace(
                go.Scatter(
                    x=df_new.index, 
                    y=df_new[c2], 
                    line=dict(color=color, width=width), 
                    name=c2
                ), 
                row=i, 
                col=1
            )

    fig.update_layout(
        height=len(tickers_list)*400, 
        width=600, 
        showlegend=False, 
        margin=dict(l=20, r=20, t=30, b=20),
        template="plotly_dark"
    )

    fig.show()
    
    return df

In [97]:
revenue_plot = create_plot_bar_line('Revenue', 'Net Income', secondary_y=False) # Cost of Revenue
eps_plot = create_plot_bar_line('EPS (Basic)', 'EPS Growth', y2perc=True, bar_color='#7eb37a')
ebitda_plot = create_plot_bar_line('EBITDA', 'EBITDA Margin', y2perc=True, bar_color='#805d67')
shares_plot = create_plot_bar_line('Shares Outstanding (Basic)', 'Shares Change', y2perc=True, bar_color='#ea7726') 
fcf_plot = create_plot_bar_line('Free Cash Flow', 'Free Cash Flow Per Share', bar_color='#8d8b55')
profit_plot = create_plot_bar_line('Gross Profit', 'Operating Expenses', secondary_y=False, bar_color='#7c459c')
dividends_plot = create_plot_bar_line('Dividend Per Share', 'Dividend Growth', y2perc=True, bar_color='#03c03c')
margins_plot = create_line_plot(y=['Gross Margin', 'Operating Margin', 'Profit Margin', 'Free Cash Flow Margin'], title='Margins')
ema_plot = create_ema_plot([STOCK], emas=[10, 20, 30, 40, 50])

[*********************100%***********************]  1 of 1 completed


In [98]:
expenses_plot = px.bar(data_frame=df_expenses)
expenses_plot.update_layout(
    xaxis = dict(title=None),
    yaxis = dict(title=None),
    height=400, 
    width=600, 
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y = -0.35,
        xanchor="left",
        title= None
    ),
    margin=dict(l=20, r=20, t=30, b=20),
    template="plotly_dark",
    title='Expenses'
)

In [13]:
# crypto_df = create_ema_plot(CRYPTO_LIST, emas=[10, 20, 30, 40, 50])
# stocks_df = create_ema_plot(STOCKS_LIST, emas=[10, 20, 30, 40, 50])

Seeking Alpha

In [6]:
div_list_seeking_alpha = [
    'div_growth_category_avg_5y',
    'dps_yoy_avg_5y',
    'dividend_per_share_change_dislpay_avg_5y',
    'dividend_lt_fwd_growth_avg_5y',
    'div_grow_rate3_avg_5y',
    'div_grow_rate5_avg_5y',
    'div_grow_rate10_avg_5y',
    'revenue_change_display_avg_5y',
    'eps_change_display_avg_5y',
    'fcf_per_share_change_display_avg_5y',
    'ebitda_change_display_avg_5y',
    'ebit_change_display_avg_5y',
    'return_on_net_tangible_assets_avg_5y',
    'log_of_unadjusted_stock_price_avg_5y',
    'coefficient_of_variation_90d_avg_5y',
    'degree_of_operating_leverage_ttm_avg_5y',
    'div_growth_category',
    'dps_yoy',
    'div_rate',
    'dividend_per_share_change_dislpay',
    'dividend_lt_fwd_growth',
    'div_grow_rate3',
    'div_grow_rate5',
    'div_grow_rate10',
    'revenue_change_display',
    'eps_change_display',
    'fcf_per_share_change_display',
    'ebitda_change_display',
    'ebit_change_display',
    'return_on_net_tangible_assets',
    'log_of_unadjusted_stock_price',
    'coefficient_of_variation_90d',
    'degree_of_operating_leverage_ttm',
    'div_yield_category',
    'div_yield_4y',
    'dividend_yield',
    'div_yield_fwd',
    'yld_on_cost_1y',
    'yld_on_cost_3y',
    'yld_on_cost_5y',
    'earnings_yield',
    'earn_yield_gaap_fy1',
    'oper_income_market_cap',
    'oper_income_fy1_market_cap',
    'fcf_yield',
    'fcf_yield_fy1'
]

In [7]:
# div_growth
url = "https://seekingalpha.com/api/v3/metrics"
querystring = {
    "filter[fields][]":div_list_seeking_alpha,
    "filter[slugs]":f"{STOCK.lower()}",
    "minified":"false"
}
headers_div_growth = {
    "cookie": "machine_cookie=9717268612629; machine_cookie_ts=1671790378",
    "authority": "seekingalpha.com",
    "referer": f"https://seekingalpha.com/symbol/{STOCK}/dividends/dividend-growth",
    "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36"
}
response = requests.request("GET", url, headers=headers_div_growth, params=querystring).json()
result = {}
for item in response["data"]:
    field = next(filter(lambda included: included["id"] == item["relationships"]["metric_type"]["data"]["id"], response["included"]))["attributes"]["field"]
    result[field] = item["attributes"]["value"]
result

{'revenue_change_display': 9.24069,
 'ebit_change_display': 9.980835,
 'eps_change_display': 13.0139065,
 'dividend_yield': 2.3930657,
 'fcf_per_share_change_display': 4.147228,
 'ebitda_change_display': 7.24649,
 'div_yield_4y': 2.0253594,
 'div_yield_fwd': 2.449595,
 'dividend_per_share_change_dislpay': 11.135535,
 'dividend_lt_fwd_growth': 3.0213392,
 'div_grow_rate10': 15.09843964909632,
 'div_grow_rate3': 11.14433440926634,
 'div_grow_rate5': 15.420355146498,
 'dps_yoy': 21.5686,
 'earnings_yield': 5.2807612,
 'oper_income_market_cap': 7.338628,
 'oper_income_fy1_market_cap': 7.4262395,
 'fcf_yield': 4.4275823,
 'fcf_yield_fy1': 4.611892,
 'yld_on_cost_1y': 2.0297267,
 'yld_on_cost_3y': 2.8548949,
 'yld_on_cost_5y': 3.6363637,
 'revenue_change_display_avg_5y': 3.529854,
 'ebit_change_display_avg_5y': 7.066011,
 'eps_change_display_avg_5y': 14.03045,
 'fcf_per_share_change_display_avg_5y': 9.820501,
 'ebitda_change_display_avg_5y': 6.013882,
 'dividend_per_share_change_dislpay_avg_

In [8]:
# sector_metrics
url = f"https://seekingalpha.com/api/v3/symbols/{STOCK.lower()}/sector_metrics"
querystring = {"filter[fields][]":div_list_seeking_alpha}
response = requests.request("GET", url, headers=headers_div_growth, params=querystring).json()
result = {}
for item in response['data']:
    value = item['attributes']['value']
    metric_id = item['relationships']['metric_type']['data']['id']
    for included in response['included']:
        if included['id'] == metric_id:
            result[included['attributes']['field']] = value
result

{'coefficient_of_variation_90d': 7.17819,
 'degree_of_operating_leverage_ttm': 1.2262,
 'dividend_lt_fwd_growth': 3.12889,
 'dividend_per_share_change_dislpay': 6.47456,
 'dividend_yield': 1.54321,
 'div_grow_rate10': 8.03866,
 'div_grow_rate3': 6.16804,
 'div_grow_rate5': 7.48577,
 'div_yield_4y': 1.6875,
 'div_yield_fwd': 1.54321,
 'dps_yoy': 8.4615,
 'earnings_yield': 5.80043,
 'earn_yield_gaap_fy1': 5.28871,
 'ebitda_change_display': 12.0111,
 'ebit_change_display': 14.7576,
 'eps_change_display': 15.1612,
 'fcf_per_share_change_display': 4.87311,
 'fcf_yield': 3.83999,
 'fcf_yield_fy1': 4.79816,
 'log_of_unadjusted_stock_price': 4.12775,
 'oper_income_fy1_market_cap': 8.608,
 'oper_income_market_cap': 7.55822,
 'return_on_net_tangible_assets': 22.0509,
 'revenue_change_display': 10.3866,
 'yld_on_cost_1y': 1.47815,
 'yld_on_cost_3y': 1.91449,
 'yld_on_cost_5y': 1.95231}

In [9]:
# ticker_metric_grades
url = "https://seekingalpha.com/api/v3/ticker_metric_grades"
querystring = {
    "filter[fields][]":div_list_seeking_alpha,
    "filter[slugs]":f"{STOCK.lower()}",
    "filter[algos][]":["main_quant","dividends"],
    "minified":"false"
}
response = requests.request("GET", url, headers=headers_div_growth, params=querystring).json()
result = {}
for item in response['data']:
    grade = item['attributes']['grade']
    metric_id = item['relationships']['metric_type']['data']['id']
    for included in response['included']:
        if included['id'] == metric_id:
            result[included['attributes']['field']] = grade
result

{'coefficient_of_variation_90d': 5,
 'degree_of_operating_leverage_ttm': 8,
 'dividend_lt_fwd_growth': 7,
 'dividend_per_share_change_dislpay': 4,
 'dividend_yield': 4,
 'div_growth_category': 2,
 'div_grow_rate10': 3,
 'div_grow_rate3': 4,
 'div_grow_rate5': 3,
 'div_yield_4y': 6,
 'div_yield_category': 4,
 'div_yield_fwd': 4,
 'dps_yoy': 3,
 'earnings_yield': 8,
 'earn_yield_gaap_fy1': 7,
 'ebitda_change_display': 10,
 'ebit_change_display': 9,
 'eps_change_display': 8,
 'fcf_per_share_change_display': 7,
 'fcf_yield': 6,
 'fcf_yield_fy1': 8,
 'log_of_unadjusted_stock_price': 1,
 'oper_income_fy1_market_cap': 8,
 'oper_income_market_cap': 8,
 'return_on_net_tangible_assets': 10,
 'revenue_change_display': 8,
 'yld_on_cost_1y': 5,
 'yld_on_cost_3y': 4,
 'yld_on_cost_5y': 2}

In [10]:
# symbol_data
url = "https://seekingalpha.com/api/v3/symbol_data"
querystring = {"fields[]":"dividend_cagrs", "slugs":f"{STOCK.lower()}"}
response = requests.request("GET", url, headers=headers_div_growth, params=querystring).json()
result = response['data'][0]['attributes'] #['dividendCagrs']
result

{'dividendCagrs': {'2021': 18.4149184149184,
  '2020': 14.4237016756347,
  '2019': 11.1443344092664,
  '2018': 13.5103784006733,
  '2017': 15.420355146498,
  '2016': 14.494913767227,
  '2015': 12.6990220229879,
  '2014': 13.0065959705479,
  '2013': 14.6862448600785,
  '2012': 15.0984396490963,
  '2011': 16.2990149466791,
  '2010': 18.613794951089,
  '2009': 18.818018106247,
  '2008': 18.1810074926415,
  '2007': 19.0280071360972,
  '2006': 19.3428496362036,
  '2005': 18.1079055538621,
  '2004': 17.0209096859428,
  '2003': 17.2378421332851,
  '2002': 17.3389724036112,
  '2001': 16.6532502028945,
  '2000': 15.8393409551507,
  '1999': 15.1011669698371,
  '1998': 14.4286404930321,
  '1997': 10.3813776892081,
  '1996': 9.9628459786314,
  '1995': 9.57673171803881,
  '1994': 9.35799966910695,
  '1993': 9.30363010039881,
  '1992': 9.27509145686989,
  '1991': 9.26013074401912,
  '1990': 9.29431684527522,
  '1989': 10.1358273781783}}

In [46]:
# dividend yield tooltips
url = "https://seekingalpha.com/api/v3/tooltips"

querystring = {"filter[path]" : f"/symbol/{STOCK}"}
response1 = requests.request("GET", url, headers=headers_div_growth, params=querystring).json()

tooltips = {}

for item in response1['data']:
    term = item['attributes']['term']
    content = item['attributes']['content']
    response1[term] = content

querystring = {"filter[path]" : f"/symbol/{STOCK}/dividends/yield"}
response2 = requests.request("GET", url, headers=headers_div_growth, params=querystring).json()

for item in response2['data']:
    term = item['attributes']['term']
    content = item['attributes']['content']
    tooltips[term] = content
    
tooltips

{'Dividend Yield (TTM)': 'The sum of actual ordinary cash dividend payments in the last 12 months divided by the last close price. Special dividends are not included in the yield. Yield (TTM) is calculated for issuers where dividend payment consistency is low, making Yield (FWD) predictions less reliable.',
 'Quant Ranking': 'Quant Ratings are an objective, quantitative view of each stock relative to other stocks in the same sector. The ratings provide an instant summary of the available data about the stock, including the company’s financial results, the stock’s trading history, and sell-side analysts’ estimates of future revenue and earnings. <br><br> The Quant Rankings are based on the underlying rating of each stock and provides the overall rank order of the stock within the sector and overall stock universe. <br><br>For more details, see <a href="/article/4263303-quant-ratings-factor-grades-faq" target="_blank">Quant Ranking and Factor Grades FAQ</a>',
 '4 Year Average Dividend Yi

In [40]:
# articles
url = f"https://seekingalpha.com/api/v3/symbols/{STOCK.lower()}/author_ratings"
querystring = {"include" : ['article']}
response = requests.request("GET", url, headers=headers_div_growth, params=querystring).json()
result = {}
for item in response['data']:
    article_id = item['relationships']['article']['data']['id']
    for item in response['included']:
        if item['id'] == article_id:
            title = item['attributes']['title']
            pro_publish_on = item['attributes']['proPublishOn']
            result[pro_publish_on] = title
result

{'2022-12-14T10:10:50-05:00': "Domino's Pizza Could Deliver Superior Returns As Bear Case Crumbles",
 '2022-12-13T18:57:46-05:00': "Domino's Pizza: A Great Short Opportunity For 2023 As Margin And Sales Pressures Mount",
 '2022-12-12T03:25:20-05:00': "Domino's: A Buy If You're Looking For More Dough",
 '2022-10-24T13:28:19-04:00': "Domino's Pizza Is One Of Our Favorite Ideas",
 '2022-01-14T10:53:22-05:00': "Domino's Showcases Quantitative Investing Pitfalls, Stock Continues To Defy Gravity",
 '2022-10-13T19:01:50-04:00': "I Hope Domino's Pizza Delivers",
 '2022-10-13T12:17:38-04:00': "Domino's Pizza Q3 Results Point To A Positive Turn",
 '2022-10-13T11:50:21-04:00': "Domino's Pizza: Down 45% From Highs, Pain Is Baked In",
 '2022-10-10T01:12:44-04:00': "Domino's: Grab A Slice As Shares Near Critical Support Heading Into Earnings",
 '2022-09-12T03:39:07-04:00': "Domino's: Tough Road Ahead",
 '2022-07-29T10:00:00-04:00': "Domino's Pizza: Long-Term Outlook Remains Solid Despite Weak Second

In [ ]:
"""quant rating"""
# https://seekingalpha.com/api/v3/symbols/dpz/rating/histories?page[number]=1
# https://seekingalpha.com/api/v3/historical_prices?filter[ticker][slug]=dpz&&filter[as_of_date][gte]=2022-07-25&filter[as_of_date][lte]=2022-09-12&sort=as_of_date
# https://seekingalpha.com/api/v3/tooltips?filter[path]=%2Fsymbol%2FDPZ%2Fratings%2Fquant-ratings
# https://seekingalpha.com/api/v3/symbols/DPZ/rating/periods?filter[periods][]=0&filter[periods][]=3&filter[periods][]=6
# https://seekingalpha.com/api/v3/symbols/dpz/fundamentals_metrics?period_type=annual&statement_type=income-statement&target_currency=USD

In [ ]:
"""wall street ratings"""
# https://seekingalpha.com/api/v3/symbol_data/estimates?estimates_data_items=outperform%2Cbuy%2Chold%2Cunderperform%2Csell&group_by_month=true&period_type=non_periodic&return_window=4&ticker_ids=2557
# https://seekingalpha.com/api/v3/symbol_data/estimates?estimates_data_items=target_price&group_by_month=false&period_type=non_periodic&return_window=4&ticker_ids=2557
# https://seekingalpha.com/api/v3/symbol_data/estimates?estimates_data_items=target_price%2Ctarget_price_high%2Ctarget_price_low&group_by_month=false&period_type=non_periodic&return_window=1&ticker_ids=2557
# https://seekingalpha.com/api/v3/metrics?filter[fields]=authors_rating_strong_buy_count%2Cauthors_rating_buy_count%2Cauthors_rating_hold_count%2Cauthors_rating_sell_count%2Cauthors_rating_strong_sell_count%2Csell_side_rating_strong_buy_count%2Csell_side_rating_buy_count%2Csell_side_rating_hold_count%2Csell_side_rating_sell_count%2Csell_side_rating_strong_sell_count%2Cauthors_rating%2Csell_side_rating&filter[slugs]=dpz&minified=false
# https://seekingalpha.com/api/v3/symbols/DPZ/rating/periods?filter[periods][]=0&filter[periods][]=3&filter[periods][]=6

In [60]:
# income (doesn't work)
# url = "https://seekingalpha.com/api/v3/symbols/dpz/fundamentals_metrics"
# querystring = {
#     "period_type" : 'quarterly',
#     'statement_type':'income-statement',
#     'target_currency':'USD'
# }
# response = requests.request("GET", url, headers=headers_div_growth, params=querystring).json()
# response

macrotrends

In [3]:
tickers_macrotrends_dict = {}
macrotrends_list = requests.get("https://www.macrotrends.net/assets/php/ticker_search_list.php?_=1673472383864").json()

for e in macrotrends_list:
    url_link = list(e.values())[1]
    ticker = list(e.values())[0].split(' - ')[0]
    tickers_macrotrends_dict[ticker] = url_link
# tickers_macrotrends_dict

In [4]:
# url = f"https://www.macrotrends.net/stocks/charts/{tickers_macrotrends_dict[STOCK]}/balance-sheet?freq=Q"

def get_macrotrends_data(url:str):
    response = requests.get(url)
    html = response.text
    soup = BeautifulSoup(html, "html.parser")
    scripts = soup.find_all("script", type="text/javascript")

    for script in scripts:
        if 'originalData' in script.text:
            var_data = script.text

    # Use regular expressions to extract the variable
    match = re.search('var originalData = (.*);', var_data)
    data = match.group(1)
    df = pd.read_json(data)

    for i, k in enumerate(df['field_name']):
        if BeautifulSoup(k, "html.parser").find('a'):
            df.loc[i, "field_name"] = BeautifulSoup(k, "html.parser").find('a').text
        else:
            df.loc[i, "field_name"] = BeautifulSoup(k, "html.parser").find('span').text

    df = df.drop(columns=['popup_icon']).set_index('field_name').replace('', 0).astype(float)
    return df

In [6]:
# balance
balance = get_macrotrends_data(f"https://www.macrotrends.net/stocks/charts/{tickers_macrotrends_dict[STOCK]}/balance-sheet?freq=Q")
balance.head()

,2022-09-30,2022-06-30,2022-03-31,2021-12-31,2021-09-30,2021-06-30,2021-03-31,2020-12-31,2020-09-30,2020-06-30,2020-03-31,2019-12-31,2019-09-30,2019-06-30,2019-03-31,2018-12-31,2018-09-30,2018-06-30,2018-03-31,2017-12-31,2017-09-30,2017-06-30,2017-03-31,2016-12-31,2016-09-30,2016-06-30,2016-03-31,2015-12-31,2015-09-30,2015-06-30,2015-03-31,2014-12-31,2014-09-30,2014-06-30,2014-03-31,2013-12-31,2013-09-30,2013-06-30,2013-03-31,2012-12-31,2012-09-30,2012-06-30,2012-03-31,2011-12-31,2011-09-30,2011-06-30,2011-03-31,2010-12-31,2010-09-30,2010-06-30,2010-03-31,2009-12-31,2009-09-30,2009-06-30,2009-03-31
field_name,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Cash On Hand,1313.0,834.0,955.0,1006.0,1240.0,1161.0,1243.0,1859.0,2661.0,2766.0,1190.0,891.0,1310.0,1109.0,1119.0,1333.0,1900.0,1694.0,1138.0,1365.0,1937.0,1376.0,1139.0,1337.0,2239.0,2160.0,2673.0,1391.0,1078.0,2041.0,1587.0,1586.0,1887.0,1537.0,1857.0,1432.0,1366.0,1845.0,1917.0,1063.0,1130.0,1201.0,995.0,1217.0,1647.0,1055.0,1248.0,1086.0,1358.0,1317.0,1753.0,1850.0,1435.0,1656.0,1466.0
Receivables,2052.0,2052.0,1958.0,1722.0,1679.0,1666.0,1610.0,1505.0,1565.0,1388.0,1669.0,1595.0,1650.0,1809.0,1672.0,1755.0,1792.0,1634.0,1571.0,1493.0,1428.0,1357.0,1279.0,1258.0,1351.0,1373.0,1355.0,1356.0,1508.0,1513.0,1564.0,1611.0,1635.0,1666.0,1542.0,1414.0,1469.0,1468.0,1512.0,1331.0,1583.0,1497.0,1327.0,1401.0,1424.0,1427.0,1275.0,1184.0,1297.0,1250.0,1218.0,666.0,727.0,629.0,548.0
Inventory,794.0,790.0,726.0,621.0,664.0,688.0,659.0,638.0,649.0,668.0,692.0,751.0,771.0,775.0,780.0,742.0,789.0,772.0,808.0,749.0,742.0,726.0,760.0,717.0,690.0,660.0,716.0,736.0,741.0,762.0,744.0,712.0,736.0,739.0,705.0,653.0,683.0,654.0,655.0,660.0,680.0,646.0,682.0,614.0,607.0,630.0,582.0,534.0,504.0,496.0,506.0,475.0,490.0,507.0,446.0
Pre-Paid Expenses,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Other Current Assets,246.0,300.0,237.0,202.0,227.0,249.0,207.0,212.0,255.0,261.0,253.0,222.0,342.0,393.0,382.0,333.0,335.0,394.0,380.0,399.0,342.0,410.0,410.0,284.0,322.0,373.0,339.0,647.0,397.0,432.0,334.0,492.0,304.0,322.0,316.0,223.0,316.0,334.0,311.0,297.0,260.0,288.0,247.0,189.0,231.0,272.0,265.0,367.0,394.0,275.0,280.0,350.0,266.0,271.0,267.0


In [123]:
# income statement
income_df = get_macrotrends_data(f"https://www.macrotrends.net/stocks/charts/{tickers_macrotrends_dict[STOCK]}/income-statement?freq=Q")
income_df.head()

,2022-09-30,2022-06-30,2022-03-31,2021-12-31,2021-09-30,2021-06-30,2021-03-31,2020-12-31,2020-09-30,2020-06-30,2020-03-31,2019-12-31,2019-09-30,2019-06-30,2019-03-31,2018-12-31,2018-09-30,2018-06-30,2018-03-31,2017-12-31,2017-09-30,2017-06-30,2017-03-31,2016-12-31,2016-09-30,2016-06-30,2016-03-31,2015-12-31,2015-09-30,2015-06-30,2015-03-31,2014-12-31,2014-09-30,2014-06-30,2014-03-31,2013-12-31,2013-09-30,2013-06-30,2013-03-31,2012-12-31,2012-09-30,2012-06-30,2012-03-31,2011-12-31,2011-09-30,2011-06-30,2011-03-31,2010-12-31,2010-09-30,2010-06-30,2010-03-31,2009-12-31,2009-09-30,2009-06-30,2009-03-31
field_name,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Revenue,6566.0,6269.0,5860.0,5733.0,5566.0,5504.0,5001.0,5141.0,4919.0,4244.0,5229.0,5212.0,5516.0,5596.0,5384.0,5757.0,5928.0,5672.0,5475.0,5450.0,5408.0,5250.0,5132.0,5168.0,5174.0,4770.0,4829.0,5208.0,5562.0,5429.0,5614.0,6153.0,6182.0,6015.0,5638.0,5630.0,5573.0,5470.0,5290.0,5250.0,5343.0,5221.0,5112.0,5108.0,5101.0,4858.0,4490.0,4410.0,4408.0,4182.0,3965.0,3754.0,3671.0,3303.0,3415.0
Cost Of Goods Sold,2836.0,2654.0,2376.0,2205.0,2094.0,1997.0,1927.0,1845.0,1817.0,1593.0,2014.0,2092.0,2212.0,2278.0,2312.0,2502.0,2553.0,2514.0,2461.0,2368.0,2302.0,2235.0,2288.0,2199.0,2149.0,2076.0,2102.0,2233.0,2340.0,2446.0,2576.0,2767.0,2819.0,2805.0,2782.0,2700.0,2650.0,2633.0,2673.0,2588.0,2610.0,2575.0,2663.0,2598.0,2615.0,2586.0,2468.0,2266.0,2165.0,2131.0,2074.0,1987.0,1878.0,1745.0,1860.0
Gross Profit,3730.0,3615.0,3484.0,3528.0,3472.0,3507.0,3074.0,3296.0,3102.0,2651.0,3215.0,3120.0,3304.0,3318.0,3072.0,3255.0,3375.0,3158.0,3014.0,3082.0,3106.0,3015.0,2844.0,2969.0,3025.0,2694.0,2727.0,2975.0,3222.0,2983.0,3038.0,3386.0,3363.0,3210.0,2856.0,2930.0,2923.0,2837.0,2617.0,2662.0,2733.0,2646.0,2449.0,2510.0,2486.0,2272.0,2022.0,2144.0,2243.0,2051.0,1891.0,1767.0,1793.0,1558.0,1555.0
Research And Development Expenses,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
SG&A Expenses,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [124]:
# cash flow statement
cash_flow_df = get_macrotrends_data(f"https://www.macrotrends.net/stocks/charts/{tickers_macrotrends_dict[STOCK]}/cash-flow-statement?freq=Q")
cash_flow_df.head()

,2022-09-30,2022-06-30,2022-03-31,2021-12-31,2021-09-30,2021-06-30,2021-03-31,2020-12-31,2020-09-30,2020-06-30,2020-03-31,2019-12-31,2019-09-30,2019-06-30,2019-03-31,2018-12-31,2018-09-30,2018-06-30,2018-03-31,2017-12-31,2017-09-30,2017-06-30,2017-03-31,2016-12-31,2016-09-30,2016-06-30,2016-03-31,2015-12-31,2015-09-30,2015-06-30,2015-03-31,2014-12-31,2014-09-30,2014-06-30,2014-03-31,2013-12-31,2013-09-30,2013-06-30,2013-03-31,2012-12-31,2012-09-30,2012-06-30,2012-03-31,2011-12-31,2011-09-30,2011-06-30,2011-03-31,2010-12-31,2010-09-30,2010-06-30,2010-03-31,2009-12-31,2009-09-30,2009-06-30,2009-03-31
field_name,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Net Income/Loss,1895.0,1835.0,1630.0,1711.0,1673.0,1798.0,1341.0,1380.0,1363.0,1132.0,1474.0,1403.0,1555.0,1570.0,1391.0,1554.0,1593.0,1509.0,1310.0,7278.0,1194.0,1168.0,1072.0,1144.0,1131.0,979.0,979.0,1117.0,1300.0,1204.0,1151.0,1431.0,1370.0,1291.0,1088.0,1174.0,1151.0,1106.0,957.0,1036.0,1042.0,1002.0,863.0,964.0,904.0,785.0,639.0,775.0,778.0,711.0,516.0,549.0,514.0,465.0,362.0
Total Depreciation And Amortization - Cash Flow,563.0,559.0,555.0,556.0,553.0,550.0,549.0,557.0,555.0,551.0,547.0,559.0,557.0,551.0,549.0,555.0,547.0,546.0,543.0,532.0,528.0,525.0,520.0,520.0,512.0,504.0,502.0,517.0,507.0,497.0,491.0,489.0,481.0,470.0,464.0,458.0,447.0,438.0,434.0,453.0,447.0,433.0,427.0,413.0,408.0,401.0,395.0,380.0,372.0,368.0,367.0,373.0,363.0,350.0,341.0
Other Non-Cash Items,87.0,33.0,60.0,-133.0,61.0,74.0,54.0,-102.0,156.0,80.0,91.0,249.0,88.0,106.0,103.0,-4.0,108.0,92.0,112.0,-5692.0,216.0,153.0,145.0,218.0,170.0,180.0,169.0,404.0,108.0,8.0,101.0,552.0,130.0,69.0,75.0,54.0,272.0,181.0,184.0,397.0,216.0,150.0,124.0,265.0,262.0,287.0,172.0,251.0,310.0,61.0,50.0,384.0,95.0,64.0,13.0
Total Non-Cash Items,650.0,592.0,615.0,423.0,614.0,624.0,603.0,455.0,711.0,631.0,638.0,808.0,645.0,657.0,652.0,551.0,655.0,638.0,655.0,-5160.0,744.0,678.0,665.0,738.0,682.0,684.0,671.0,921.0,615.0,505.0,592.0,1041.0,611.0,539.0,539.0,512.0,719.0,619.0,618.0,850.0,663.0,583.0,551.0,678.0,670.0,688.0,567.0,631.0,682.0,429.0,417.0,757.0,458.0,414.0,354.0
Change In Accounts Receivable,0.0,-94.0,-236.0,-43.0,-13.0,-56.0,-105.0,60.0,-177.0,281.0,-74.0,55.0,159.0,-137.0,83.0,37.0,-158.0,-63.0,-78.0,-65.0,-71.0,-78.0,-21.0,93.0,22.0,-18.0,1.0,152.0,5.0,51.0,47.0,24.0,31.0,-124.0,-128.0,55.0,-1.0,44.0,-181.0,252.0,-86.0,-170.0,74.0,23.0,3.0,-152.0,-91.0,113.0,-47.0,-32.0,-552.0,61.0,-98.0,-81.0,46.0


In [125]:
# financial ratios
fin_ratios_df = get_macrotrends_data(f"https://www.macrotrends.net/stocks/charts/{tickers_macrotrends_dict[STOCK]}/financial-ratios?freq=Q")
fin_ratios_df.head()

,2022-09-30,2022-06-30,2022-03-31,2021-12-31,2021-09-30,2021-06-30,2021-03-31,2020-12-31,2020-09-30,2020-06-30,2020-03-31,2019-12-31,2019-09-30,2019-06-30,2019-03-31,2018-12-31,2018-09-30,2018-06-30,2018-03-31,2017-12-31,2017-09-30,2017-06-30,2017-03-31,2016-12-31,2016-09-30,2016-06-30,2016-03-31,2015-12-31,2015-09-30,2015-06-30,2015-03-31,2014-12-31,2014-09-30,2014-06-30,2014-03-31,2013-12-31,2013-09-30,2013-06-30,2013-03-31,2012-12-31,2012-09-30,2012-06-30,2012-03-31,2011-12-31,2011-09-30,2011-06-30,2011-03-31,2010-12-31,2010-09-30,2010-06-30,2010-03-31,2009-12-31,2009-09-30,2009-06-30,2009-03-31
field_name,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Current Ratio,0.7712,0.6624,0.7096,0.6182,0.7455,0.8639,0.7635,1.0098,1.0597,0.9629,0.8197,0.7950,0.8879,0.7213,0.7512,0.8999,1.0634,1.0279,0.8211,1.0170,1.1613,1.1359,0.9324,0.9879,1.4130,1.4753,1.5464,1.2882,1.1237,1.4600,1.3212,1.1692,1.2471,1.1922,1.1172,1.0525,1.1119,1.1877,1.2870,1.1587,1.0186,1.0100,0.9136,1.1236,1.1316,1.1732,1.2477,1.1626,1.2276,1.2198,1.4135,1.3721,1.1707,1.1807,1.1249
Long-term Debt / Capital,0.7300,0.7001,0.7206,0.6606,0.6645,0.6595,0.6071,0.6021,0.6026,0.6164,0.6225,0.5691,0.5744,0.5582,0.5689,0.5061,0.5036,0.5175,0.3916,0.3938,0.4541,0.4371,0.4205,0.4169,0.4284,0.4198,0.4194,0.3966,0.3832,0.3808,0.3608,0.3407,0.3388,0.3263,0.3097,0.2948,0.2967,0.3066,0.3161,0.3069,0.3098,0.3115,0.3007,0.3189,0.3207,0.3236,0.3332,0.3364,0.3419,0.3448,0.3551,0.3645,0.3693,0.3799,0.3665
Debt/Equity Ratio,2.8461,2.5183,2.7098,2.0994,2.1122,2.0144,1.6416,1.5762,1.6315,1.7280,1.7432,1.3901,1.4285,1.3897,1.4154,1.0964,1.0855,1.1459,0.7211,0.6817,0.8790,0.8035,0.7622,0.7529,0.7697,0.7436,0.7420,0.6860,0.6466,0.6355,0.5841,0.5386,0.5338,0.5057,0.4784,0.4512,0.4551,0.4779,0.4895,0.4526,0.4852,0.4933,0.4711,0.4794,0.5067,0.4883,0.5096,0.5203,0.5574,0.5406,0.5646,0.5862,0.5961,0.6236,0.5874
Gross Margin,56.8078,57.6647,59.4539,61.5385,62.3787,63.7173,61.4677,64.1121,63.0616,62.4647,61.4840,59.8619,59.8985,59.2924,57.0580,56.5399,56.9332,55.6770,55.0502,56.5505,57.4334,57.4286,55.4170,57.4497,58.4654,56.4780,56.4713,57.1237,57.9288,54.9457,54.1147,55.0301,54.3999,53.3666,50.6563,52.0426,52.4493,51.8647,49.4707,50.7048,51.1510,50.6800,47.9069,49.1386,48.7355,46.7682,45.0334,48.6168,50.8848,49.0435,47.6923,47.0698,48.8423,47.1692,45.5344
Operating Margin,40.1005,39.7990,40.5631,42.5606,43.6939,44.9310,39.8520,39.0197,41.2889,38.9727,40.9830,40.2916,40.5004,40.3860,36.4042,38.3881,38.2760,37.0064,35.4155,41.2294,38.3321,38.0571,34.8402,37.4613,37.8817,34.8008,34.9348,36.8280,39.6980,35.8998,35.2155,38.5666,37.6901,36.5087,32.8840,35.0444,35.2055,34.3327,30.8696,32.8571,33.4269,33.0205,29.5383,31.6562,30.9351,28.6538,25.3229,29.7732,31.7831,30.5835,24.9180,26.6116,26.1782,22.6461,19.6486


Alpha Spread

In [12]:
response = requests.get(url)

In [11]:
"""BeautifulSoup"""
url_ = f"https://www.alphaspread.com/security/nasdaq/aapl/summary"

# Reuse a session object: 
# Instead of creating a new session object for each request, you can reuse the same session object for multiple requests. 
# This can save time by not having to establish a new connection for each request.

# session_object = requests.Session()
# response = session_object.get(url_)

# response = requests.get(url)
# html = response.text
# soup = BeautifulSoup(html, "lxml")
# soup


# https://www.alphaspread.com/security/nasdaq/aapl/summary
# https://www.alphaspread.com/security/nasdaq/aapl/dcf-valuation
# https://www.alphaspread.com/security/nasdaq/aapl/relative-valuation
# https://www.alphaspread.com/security/nasdaq/aapl/analyst-estimates
# https://www.alphaspread.com/security/nasdaq/aapl/profitability
# https://www.alphaspread.com/security/nasdaq/aapl/solvency
# https://www.alphaspread.com/security/nasdaq/aapl/financials/balance-sheet
# https://www.alphaspread.com/security/nasdaq/aapl/financials/income-statement
# https://www.alphaspread.com/security/nasdaq/aapl/financials/cash-flow-statement
# https://www.alphaspread.com/security/nasdaq/aapl/financials/revenue-breakdown
# https://www.alphaspread.com/security/nasdaq/aapl/discount-rate

<!DOCTYPE html>
<html lang="en">
<head>
<!-- Google Tag Manager -->
<script>(function(w,d,s,l,i){w[l]=w[l]||[];w[l].push({'gtm.start':
new Date().getTime(),event:'gtm.js'});var f=d.getElementsByTagName(s)[0],
j=d.createElement(s),dl=l!='dataLayer'?'&l='+l:'';j.async=true;j.src=
'https://www.googletagmanager.com/gtm.js?id='+i+dl;f.parentNode.insertBefore(j,f);
})(window,document,'script','dataLayer','GTM-T2BHCDJ');</script>
<!-- End Google Tag Manager -->
<!-- Hotjar Tracking Code for www.alphaspread.com -->
<script>
    (function(h,o,t,j,a,r){
        h.hj=h.hj||function(){(h.hj.q=h.hj.q||[]).push(arguments)};
        h._hjSettings={hjid:1932821,hjsv:6};
        a=o.getElementsByTagName('head')[0];
        r=o.createElement('script');r.async=1;
        r.src=t+h._hjSettings.hjid+j+h._hjSettings.hjsv;
        a.appendChild(r);
    })(window,document,'https://static.hotjar.com/c/hotjar-','.js?sv=');
</script>
<meta charset="utf-8"/>
<meta content="lc0hJ2Jb3J9IdSZP2g4HCV9uGevpNExakNZBq08m

NASDAQ data link

In [ ]:
# https://docs.data.nasdaq.com/docs/in-depth-usage
# https://data.nasdaq.com/tools/python
f"https://data.nasdaq.com/api/v3/datasets/WIKI/{STOCK}/data.json?api_key={NASDAQ_DATA_LINK_API_KEY}"

In [114]:
"""last date on AAPL is 2018-03-27"""
# url = "https://data.nasdaq.com/api/v3/datasets/WIKI/AAPL/data.json"
# querystring = {"api_key":NASDAQ_DATA_LINK_API_KEY}
# response = requests.request("GET", url, params=querystring).json()
# ndl_df = pd.DataFrame(data=response['dataset_data']['data'], columns=response['dataset_data']['column_names'])
# ndl_df

'last date on AAPL is 2018-03-27'

In [116]:
"""last date on AAPL is 2017-10-31"""
# url = "https://data.nasdaq.com/api/v3/datatables/QUOTEMEDIA/PRICES"
# querystring = {"api_key":NASDAQ_DATA_LINK_API_KEY, 'ticker':'AAPL'}
# response = requests.request("GET", url, params=querystring).json()['datatable']
# ndl_df = pd.DataFrame(data=response['data'], columns=[i['name'] for i in response['columns']])
# ndl_df

'last date on AAPL is 2017-10-31'

# Summary

In [109]:
stocks_full_summary = pd.DataFrame()
for x in STOCKS_LIST:
    print(x)
    ticker = yq.Ticker(x)
    # ticker_yf = yf.Ticker(x)

    summary = pd.DataFrame(ticker.summary_detail)
    
    recomendations = ticker.recommendation_trend.reset_index().query('period=="0m"')[['strongBuy','buy','hold','sell','strongSell']].T
    recomendations.columns = [x]
    
    financials = pd.DataFrame(ticker.financial_data)
    key_stats = pd.DataFrame(ticker.key_stats)
    # esg_scores = pd.DataFrame(ticker.esg_scores)
    profile = pd.DataFrame(ticker.summary_profile)

    # earnings
    earnings = ticker.earning_history

    # grades
    grades = ticker.grading_history
    grades['epochGradeDate'] = pd.to_datetime(grades['epochGradeDate'])

    # income
    income_statement = ticker.income_statement().query("periodType=='12M'")

    if 'OperatingIncome' in income_statement.columns:
        income_statement['operatingMargin'] = income_statement['OperatingIncome']/income_statement['TotalRevenue']
    else:
        income_statement['operatingMargin'] = income_statement['PretaxIncome']/income_statement['TotalRevenue']
    
    if 'GrossProfit' in income_statement.columns:
        income_statement['grossMargin'] = income_statement['GrossProfit']/income_statement['TotalRevenue']
    else:
        income_statement['grossMargin'] = np.nan
    income_statement['avgGrossMarginGrowth'] = income_statement['grossMargin'].diff()
    income_statement['avgOperatingMarginGrowth'] = income_statement['operatingMargin'].diff()
    income_statement['avgNetIncomeGrowth'] = income_statement['NetIncome'].diff()/income_statement['NetIncome'].shift(1)
    income_statement['avgTotalRevenueGrowth'] = income_statement['TotalRevenue'].diff()/income_statement['TotalRevenue'].shift(1)

    # balance
    balance = ticker.balance_sheet().query('periodType=="12M"')
    balance['avgTotalAssetsGrowth'] = balance['TotalAssets'].diff()/balance['TotalAssets'].shift(1)
    balance['avgTotalDebtGrowth'] = balance['TotalDebt'].diff()/balance['TotalDebt'].shift(1)

    if 'NetDebt' in balance.columns:
        balance['avgNetDebtGrowth'] = balance['NetDebt'].diff()/balance['NetDebt'].shift(1)
    elif 'CurrentDebt' in balance.columns:
        balance['avgNetDebtGrowth'] = balance['CurrentDebt'].diff()/balance['CurrentDebt'].shift(1)
    else:
        balance['avgNetDebtGrowth'] = np.nan

    # cash flow
    cashflow = ticker.cash_flow().query('periodType=="12M"')
    cashflow['avgFreeCashFlowGrowth'] = cashflow['FreeCashFlow'].diff()/cashflow['FreeCashFlow'].shift(1)

    # final row
    row = pd.concat([profile, summary, key_stats, financials, recomendations], axis=0)
    row = row[~row.index.duplicated(keep='first')].T

    row['longName'] = ticker.price[x]['longName']
    if type(earnings['surprisePercent'][0]) != dict:
        row['avgSurprice'] = earnings['surprisePercent'].median()
        row['lastSurprice'] = earnings.iloc[-1]['surprisePercent']
        row['earning_dynamics'] = earnings.iloc[0]['epsActual'] / earnings.iloc[-1]['epsActual']
    else:
        row['avgSurprice'] = 0
        row['lastSurprice'] = 0
        row['earning_dynamics'] = 0

    row['operatingMargin'] = income_statement.iloc[-1]['operatingMargin']
    row['grossMargin'] = income_statement.iloc[-1]['grossMargin']
    row['operatingMarginStability'] = income_statement['operatingMargin'].max() - income_statement['operatingMargin'].min()
    row['avgGrossMarginGrowth'] = income_statement['avgGrossMarginGrowth'].mean()
    row['avgOperatingMarginGrowth'] = income_statement['avgOperatingMarginGrowth'].mean()
    row['avgNetIncomeGrowth'] = income_statement['avgNetIncomeGrowth'].mean()
    row['avgTotalRevenueGrowth'] = income_statement['avgTotalRevenueGrowth'].mean()
    row['netIncomeStability'] = income_statement['avgNetIncomeGrowth'].max() - income_statement['avgNetIncomeGrowth'].min()

    row['avgTotalAssetsGrowth'] = balance['avgTotalAssetsGrowth'].mean()
    row['avgTotalDebtGrowth'] = balance['avgTotalDebtGrowth'].mean()
    row['avgNetDebtGrowth'] = balance['avgNetDebtGrowth'].mean()
    row['netDebtStability'] = balance['avgNetDebtGrowth'].max() - balance['avgNetDebtGrowth'].min()

    if 'NetDebt' in balance.columns:
        row['lastNetDebt'] = balance.iloc[-1]['NetDebt']
    elif 'CurrentDebt' in balance.columns:
        row['lastNetDebt'] = balance.iloc[-1]['CurrentDebt']
    else:
        row['lastNetDebt'] = np.nan

    row['lastFreeCashFlow'] = cashflow.iloc[-1]['FreeCashFlow']
    row['avgFreeCashFlowGrowth'] = cashflow['avgFreeCashFlowGrowth'].mean()
    row['freeCashFlowStability'] = cashflow['avgFreeCashFlowGrowth'].max() - cashflow['avgFreeCashFlowGrowth'].min()

    row['modeGrade'] = grades.loc[(grades['epochGradeDate'].dt.month == dt.date.today().month)|
        (grades['epochGradeDate'].dt.month == dt.date.today().month-1),'toGrade'].mode()[0]
    row['lastGrade'] = grades.loc[grades['epochGradeDate'] == grades['epochGradeDate'].max(),'toGrade'][0]
    
    stocks_full_summary = pd.concat([stocks_full_summary, row], axis=0)

stocks_full_summary['debtToEBITDA'] = stocks_full_summary['lastNetDebt']/stocks_full_summary['ebitda']
stocks_full_summary.head(5)

AAPL
ABBV
ABR
ABT
AFL
AMD
AMGN
AMZN
ASML
ATVI
AVGO
BBY
BHP
BRK-B
CCI
COST
CSCO
CVX
GOOGL
HD
IRM
JNJ
JPM
LOW
MA
MAIN
MCD
MCO
MDT
META
MO
MRK
MS
MSFT
NFLX
NVDA
O
PEP
PFE
PLD
PYPL
SBUX
STOR
T
TGT
TROW
TSLA
TSM
UNP
UPS
V
VICI
WPC
XOM
ZTS


C:\Users\nazaire1703\AppData\Local\Temp\ipykernel_8888\3688986025.py:102: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  stocks_full_summary['debtToEBITDA'] = stocks_full_summary['lastNetDebt']/stocks_full_summary['ebitda']


,address1,city,companyOfficers,country,fullTimeEmployees,industry,longBusinessSummary,maxAge,phone,sector,state,website,zip,algorithm,ask,askSize,averageDailyVolume10Day,averageVolume,averageVolume10days,beta,bid,bidSize,coinMarketCapLink,currency,dayHigh,dayLow,dividendRate,dividendYield,exDividendDate,fiftyDayAverage,fiftyTwoWeekHigh,fiftyTwoWeekLow,fiveYearAvgDividendYield,forwardPE,fromCurrency,lastMarket,marketCap,open,payoutRatio,previousClose,priceHint,priceToSalesTrailing12Months,regularMarketDayHigh,regularMarketDayLow,regularMarketOpen,regularMarketPreviousClose,regularMarketVolume,toCurrency,tradeable,trailingAnnualDividendRate,trailingAnnualDividendYield,trailingPE,twoHundredDayAverage,volume,52WeekChange,SandP52WeekChange,bookValue,category,dateShortInterest,earningsQuarterlyGrowth,enterpriseToEbitda,enterpriseToRevenue,enterpriseValue,floatShares,forwardEps,fundFamily,heldPercentInsiders,heldPercentInstitutions,impliedSharesOutstanding,lastDividendDate,lastDividendValue,lastFiscalYearEnd,lastSplitDate,lastSplitFactor,legalType,mostRecentQuarter,netIncomeToCommon,nextFiscalYearEnd,pegRatio,priceToBook,profitMargins,sharesOutstanding,sharesPercentSharesOut,sharesShort,sharesShortPreviousMonthDate,sharesShortPriorMonth,shortPercentOfFloat,shortRatio,trailingEps,currentPrice,currentRatio,debtToEquity,earningsGrowth,ebitda,ebitdaMargins,financialCurrency,freeCashflow,grossMargins,grossProfits,numberOfAnalystOpinions,operatingCashflow,operatingMargins,quickRatio,recommendationKey,recommendationMean,returnOnAssets,returnOnEquity,revenueGrowth,revenuePerShare,targetHighPrice,targetLowPrice,targetMeanPrice,targetMedianPrice,totalCash,totalCashPerShare,totalDebt,totalRevenue,strongBuy,buy,hold,sell,strongSell,longName,avgSurprice,lastSurprice,earning_dynamics,operatingMargin,grossMargin,operatingMarginStability,avgGrossMarginGrowth,avgOperatingMarginGrowth,avgNetIncomeGrowth,avgTotalRevenueGrowth,netIncomeStability,avgTotalAssetsGrowth,avgTotalDebtGrowth,avgNetDebtGrowth,netDebtStability,lastNetDebt,lastFreeCashFlow,avgFreeCashFlowGrowth,freeCashFlowStability,modeGrade,lastGrade,address2,fax,debtToEBITDA
AAPL,One Apple Park Way,Cupertino,[],United States,164000,Consumer Electronics,"Apple Inc. designs, manufactures, and markets ...",86400,408 996 1010,Technology,CA,https://www.apple.com,95014,None,135.15,1100,87041980,87201515,87041980,1.219468,135.08,1200,None,USD,135.99,132.75,0.92,0.007,2022-11-04 01:00:00,144.5472,182.94,129.04,0.99,20.053175,None,None,2159683567616,132.98,0.1473,132.3,2,5.476871,135.99,132.75,132.98,132.3,21991855,None,False,0.9,0.006803,22.21931,152.52995,21991855,-0.246755,-0.186294,3.178,None,2022-11-30 01:00:00,0.008,16.776,5.554,2189933543424,15891414476,6.77,None,0.00072,0.60025,0,1667520000,0.23,2022-09-24 02:00:00,2020-08-31 02:00:00,4:1,None,2022-09-24 02:00:00,99802996736,2024-09-24 02:00:00,2.44,42.71869,0.2531,15908100096,0.0079,125719918,2022-10-31 01:00:00,103178670,0.0079,1.52,6.11,135.76,0.879,261.446,0.048,130541002752,0.33105,USD,90215251968,0.4331,170782000000,41,122151002112,0.30289,0.709,buy,1.9,0.21214,1.75459,0.081,24.317,214.0,122.0,176.45,176.0,48304001024,3.036,132480000000,394328014848,11,21,6,0,0,Apple Inc.,0.0485,0.016,1.627907,0.302887,0.433096,0.061414,0.018306,0.019056,0.247423,0.155218,0.610161,0.015165,0.037527,0.179139,0.183027,9.642300e+10,1.114430e+11,0.237194,0.068076,Buy,Overweight,NaN,NaN,0.738641
ABBV,1 North Waukegan Road,North Chicago,[],United States,50000,Drug Manufacturers—General,"AbbVie Inc. discovers, develops, manufactures,...",86400,847 932 7900,Healthcare,IL,https://www.abbvie.com,60064-6400,None,161.99,1200,5962840,5679307,5962840,0.672054,162.03,1100,None,USD,162.3,160.37,5.92,0.0367,2023-01-12 01:00:00,153.62,175.91,128.26,4.35,13.90086,None,None,286396514304,161.45,0.7373,160.58,2,4.953329,162.3,160.37,161.45,160.58,496440,None,False,5.64,0.035123,21.592669,150.02975,496440,0.216055,-0.186294,9.044,None,2022-11-30 01:00:00,0.242,11.535,5.945,3437

In [118]:
stocks_full_summary.loc[stocks_full_summary['earningsGrowth'].isna()]

,address1,city,companyOfficers,country,fullTimeEmployees,industry,longBusinessSummary,maxAge,phone,sector,state,website,zip,algorithm,ask,askSize,averageDailyVolume10Day,averageVolume,averageVolume10days,beta,bid,bidSize,coinMarketCapLink,currency,dayHigh,dayLow,dividendRate,dividendYield,exDividendDate,fiftyDayAverage,fiftyTwoWeekHigh,fiftyTwoWeekLow,fiveYearAvgDividendYield,forwardPE,fromCurrency,lastMarket,marketCap,open,payoutRatio,previousClose,priceHint,priceToSalesTrailing12Months,regularMarketDayHigh,regularMarketDayLow,regularMarketOpen,regularMarketPreviousClose,regularMarketVolume,toCurrency,tradeable,trailingAnnualDividendRate,trailingAnnualDividendYield,trailingPE,twoHundredDayAverage,volume,52WeekChange,SandP52WeekChange,bookValue,category,dateShortInterest,earningsQuarterlyGrowth,enterpriseToEbitda,enterpriseToRevenue,enterpriseValue,floatShares,forwardEps,fundFamily,heldPercentInsiders,heldPercentInstitutions,impliedSharesOutstanding,lastDividendDate,lastDividendValue,lastFiscalYearEnd,lastSplitDate,lastSplitFactor,legalType,mostRecentQuarter,netIncomeToCommon,nextFiscalYearEnd,pegRatio,priceToBook,profitMargins,sharesOutstanding,sharesPercentSharesOut,sharesShort,sharesShortPreviousMonthDate,sharesShortPriorMonth,shortPercentOfFloat,shortRatio,trailingEps,currentPrice,currentRatio,debtToEquity,earningsGrowth,ebitda,ebitdaMargins,financialCurrency,freeCashflow,grossMargins,grossProfits,numberOfAnalystOpinions,operatingCashflow,operatingMargins,quickRatio,recommendationKey,recommendationMean,returnOnAssets,returnOnEquity,revenueGrowth,revenuePerShare,targetHighPrice,targetLowPrice,targetMeanPrice,targetMedianPrice,totalCash,totalCashPerShare,totalDebt,totalRevenue,strongBuy,buy,hold,sell,strongSell,longName,avgSurprice,lastSurprice,earning_dynamics,operatingMargin,grossMargin,operatingMarginStability,avgGrossMarginGrowth,avgOperatingMarginGrowth,avgNetIncomeGrowth,avgTotalRevenueGrowth,netIncomeStability,avgTotalAssetsGrowth,avgTotalDebtGrowth,avgNetDebtGrowth,netDebtStability,lastNetDebt,lastFreeCashFlow,avgFreeCashFlowGrowth,freeCashFlowStability,modeGrade,lastGrade,address2,fax,debtToEBITDA,TOTAL_SCORE
BRK-B,3555 Farnam Street,Omaha,[],United States,372000,Insurance—Diversified,"Berkshire Hathaway Inc., through its subsidiar...",86400,402 346 1400,Financial Services,NE,https://www.berkshirehathaway.com,68131,None,306.79,800,4361740,4174663,4361740,0.919899,306.56,900,None,USD,307.82,304.32,NaN,NaN,NaN,298.4184,362.1,259.85,NaN,20.094444,None,None,678343606272,304.38,0.0,302.0,2,2.293856,307.82,304.32,304.38,302.0,646682,None,False,0.0,0.0,59.01056,301.9704,646682,0.02578,-0.186294,310560.2,None,2022-11-30 01:00:00,NaN,1.638,0.055,16294445056,1205911,15.3,None,0.00441,0.65289,0,NaN,NaN,2021-12-31 01:00:00,2010-01-21 01:00:00,50:1,None,2022-09-30 02:00:00,-1336999936,2023-12-31 01:00:00,0.89,0.00099,-0.00452,1304380032,0.0024,5185763,2022-10-31 01:00:00,6398341,0.0045,1.32,5.21,307.445,1.459,25.12,NaN,9946999808,0.03364,USD,-23768750080,-0.00304,114863000000,1,34817998848,-0.00304,1.245,hold,2.7,-0.00062,-0.00102,0.09,200580.47,362.0,362.0,362.0,362.0,108957999104,74383.305,116495998976,295721992192,1,3,2,0,0,Berkshire Hathaway Inc.,0.2000,0.322,0.926346,0.314931,NaN,0.297179,NaN,0.099060,6.627330,0.188514,19.725686,0.107050,0.056210,-0.093199,1.379631,2.607800e+10,2.614500e+10,0.049562,0.201502,Equal-Weight,Equal-Weight,NaN,402 346 3375,2.621695,NaN
MO,6601 West Broad Street,Richmond,[],United States,6000,Tobacco,"Altria Group, Inc., through its subsidiaries, ...",86400,804 274 2200,Consumer Defensive,VA,https://www.altria.com,23230,None,45.59,1000,8591000,8564788,8591000,0.67852,45.58,800,None,USD,45.76,45.4009,3.76,0.0808,2022-12-21 01:00:00,45.5256,57.05,40.35,6.83,9.034862,None,None,81607720960,45.61,1.4163,45.5,2,3.944117,45.76,45.4009,45.61,45.5,1797544,None,False,3.64,0.08,17.718172,47.3383,1797544,-0.009386,-0.186294,-2.359,None,2022-11-30 01:00:00,NaN,8.631,5.183,107251556352,1790326680,5.04,None,0.00

In [120]:
stocks_full_summary[['grossMargins','grossMargin']]

,grossMargins,grossMargin
AAPL,0.4331,0.433096
ABBV,0.69832,0.689556
ABR,0.91626,NaN
ABT,0.58163,0.569658
AFL,0.42594,NaN
AMD,0.5095,0.482475
AMGN,0.7578,0.751569
AMZN,0.43044,0.141149
ASML,0.51142,0.527054
ATVI,0.71129,0.736794


In [128]:
# stocks_full_summary['TOTAL_SCORE'] = \
#     stocks_full_summary['marketCap']\
#     *stocks_full_summary['dividendYield']\
#     *stocks_full_summary['avgNetIncomeGrowth']\
#     *stocks_full_summary['avgFreeCashFlowGrowth']\
#     *stocks_full_summary['operatingMargins']*stocks_full_summary['operatingMargin']*stocks_full_summary['avgOperatingMarginGrowth']\
#     *stocks_full_summary['grossMargins']*stocks_full_summary['grossMargin']*stocks_full_summary['avgGrossMarginGrowth']\
#     *(stocks_full_summary['forwardEps'] / stocks_full_summary['trailingEps'])\
#     *stocks_full_summary['avgSurprice']\
#     *stocks_full_summary['totalCashPerShare']\
#     *stocks_full_summary['payoutRatio']
    # stocks_full_summary['freeCashflow'] * stocks_full_summary['lastFreeCashFlow'] * stocks_full_summary['avgFreeCashFlowGrowth']\
    # *stocks_full_summary['operatingCashflow']\
    # *stocks_full_summary['operatingMargins']*stocks_full_summary['operatingMargin']*stocks_full_summary['avgOperatingMarginGrowth']\
    # *stocks_full_summary['grossMargins']*stocks_full_summary['grossMargin']*stocks_full_summary['avgGrossMarginGrowth']\
    # *stocks_full_summary['dividendYield'] * stocks_full_summary['fiveYearAvgDividendYield'] * stocks_full_summary['trailingAnnualDividendYield']\
    # *(stocks_full_summary['forwardPE'] / stocks_full_summary['trailingPE'])\
    # *(stocks_full_summary['forwardEps'] / stocks_full_summary['trailingEps'])\
    # /(stocks_full_summary['avgNetDebtGrowth'] * stocks_full_summary['debtToEquity'] * stocks_full_summary['debtToEBITDA'])\
    # *(stocks_full_summary['returnOnAssets']*stocks_full_summary['returnOnEquity'])\
    # *(stocks_full_summary['revenueGrowth']*stocks_full_summary['revenuePerShare'])\
    # *(stocks_full_summary['ebitda']*stocks_full_summary['ebitdaMargins'])\
    # *(stocks_full_summary['avgNetIncomeGrowth'] * stocks_full_summary['avgTotalRevenueGrowth'])\
    # *(stocks_full_summary['avgTotalAssetsGrowth'] - stocks_full_summary['avgTotalDebtGrowth'])\
    # *stocks_full_summary['avgSurprice']\
    # *stocks_full_summary['totalCashPerShare']\
    # *stocks_full_summary['marketCap']\
    # *stocks_full_summary['payoutRatio']
stocks_full_summary[['longName','marketCap']].sort_values(by='marketCap', ascending=False)

,longName,marketCap
AAPL,Apple Inc.,2159683567616
MSFT,Microsoft Corporation,1817474367488
GOOGL,Alphabet Inc.,1162320281600
AMZN,"Amazon.com, Inc.",884334395392
BRK-B,Berkshire Hathaway Inc.,678343606272
JNJ,Johnson & Johnson,464148594688
XOM,Exxon Mobil Corporation,446443225088
TSLA,"Tesla, Inc.",438737797120
V,Visa Inc.,438432038912
NVDA,NVIDIA Corporation,411329527808


In [117]:
isna_stocks = stocks_full_summary.isna().sum()
isna_stocks

address1              0
city                  0
companyOfficers       0
country               0
fullTimeEmployees     0
                     ..
lastGrade             0
address2             43
fax                  46
debtToEBITDA         11
TOTAL_SCORE          13
Length: 148, dtype: int64

In [ ]:
# cashflow = ticker.cash_flow().query('periodType=="12M"')
# cashflow.FreeCashFlow

In [122]:
stocks_summary = stocks_full_summary[[
    'longName', 
    'sector',
    'industry',
    'marketCap',
    'currentPrice',
    'averageVolume',
    'earningsGrowth',
    'revenueGrowth',
    'trailingPE',
    'forwardPE',
    'priceToSalesTrailing12Months',
    'dividendYield',
    'fiveYearAvgDividendYield',
    'trailingAnnualDividendYield',
    'exDividendDate',
    'freeCashflow',
    'revenuePerShare',
    'targetHighPrice',
    'targetLowPrice',
    'targetMeanPrice',
    'targetMedianPrice',
    'recommendationKey',
    'modeGrade',
    'strongBuy','buy','hold','sell','strongSell'
    ]].copy()


stocks_summary['exDividendDate'] = pd.to_datetime(stocks_summary['exDividendDate']).dt.strftime('%Y-%m-%d')

stocks_summary['marketCap'] = stocks_summary['marketCap']/1e9
stocks_summary['freeCashflow'] = stocks_summary['marketCap']/1e9
stocks_summary['averageVolume'] = stocks_summary['averageVolume']/1e6
stocks_summary['targetHighPrice'] = stocks_summary['targetHighPrice']/stocks_summary['currentPrice']-1
stocks_summary['targetLowPrice'] = stocks_summary['targetLowPrice']/stocks_summary['currentPrice']-1
stocks_summary['targetMeanPrice'] = stocks_summary['targetMeanPrice']/stocks_summary['currentPrice']-1
stocks_summary['targetMedianPrice'] = stocks_summary['targetMedianPrice']/stocks_summary['currentPrice']-1
stocks_summary

,longName,sector,industry,marketCap,currentPrice,averageVolume,earningsGrowth,revenueGrowth,trailingPE,forwardPE,priceToSalesTrailing12Months,dividendYield,fiveYearAvgDividendYield,trailingAnnualDividendYield,exDividendDate,freeCashflow,revenuePerShare,targetHighPrice,targetLowPrice,targetMeanPrice,targetMedianPrice,recommendationKey,modeGrade,strongBuy,buy,hold,sell,strongSell
AAPL,Apple Inc.,Technology,Consumer Electronics,2159.683568,135.76,87.201515,0.048,0.081,22.21931,20.053175,5.476871,0.007,0.99,0.006803,2022-11-04,0.000002,24.317,0.576311,-0.101355,0.29972,0.296405,buy,Buy,11,21,6,0,0
ABBV,AbbVie Inc.,Healthcare,Drug Manufacturers—General,286.396514,161.945,5.679307,0.242,0.033,21.592669,13.90086,4.953329,0.0367,4.35,0.035123,2023-01-12,0.0,32.638,0.234987,-0.166384,-0.006576,-0.01201,buy,Buy,4,6,10,0,0
ABR,"Arbor Realty Trust, Inc.",Real Estate,REIT—Mortgage,2.343018,13.66,2.206565,-0.285,-0.054,7.189474,9.046357,3.608657,0.119,9.12,0.114925,2022-11-17,0.0,4.091,0.354319,0.061493,0.207906,0.24451,buy,Market Outperform,0,2,1,0,0
ABT,Abbott Laboratories,Healthcare,Medical Devices,188.20096,107.94,5.985533,-0.308,-0.047,24.36569,24.476192,4.179457,0.0191,1.53,0.017654,2023-01-12,0.0,25.592,0.334074,-0.166203,0.082824,0.088568,buy,Outperform,7,6,7,0,0
AFL,Aflac Incorporated,Financial Services,Insurance—Life,44.669317,71.84,2.715812,0.917,-0.08,9.1867,13.157509,2.134428,0.0244,2.35,0.021653,2023-02-14,0.0,32.465,0.113586,-0.24833,-0.056654,-0.039532,hold,Equal-Weight,1,2,10,1,1
AMD,"Advanced Micro Devices, Inc.",Technology,Semiconductors,108.592439,67.34,80.917463,-0.947,0.29,41.8323,18.452053,4.756985,NaN,NaN,0.0,1995-04-27,0.0,15.646,1.970003,-0.108999,0.349421,0.299376,buy,Buy,5,6,15,4,1
AMGN,Amgen Inc.,Healthcare,Drug Manufacturers—General,142.153449,266.415,2.890631,0.202,-0.008,21.381622,14.424202,5.398916,0.0322,2.82,0.028631,2023-02-14,0.0,48.29,0.219901,-0.324362,0.00092,0.015333,hold,Overweight,3,7,15,0,0
AMZN,"Amazon.com, Inc.",Consumer Cyclical,Internet Retail,884.334395,86.685,79.831473,-0.097,0.147,79.52752,52.21988,1.760952,NaN,NaN,0.0,NaN,0.000001,49.349,1.685009,-0.077118,0.613774,0.580435,buy,Buy,15,28,3,1,0
ASML,ASML Holding N.V.,Technology,Semiconductor Equipment & Materials,238.440071,576.9219,1.402849,0.007,0.102,38.90235,29.13747,12.085888,0.0116,0.83,0.009003,2022-11-03,0.0,49.224,0.565567,-0.181865,0.196678,0.148232,buy,Overweight,2,3,2,0,0
ATVI,"Activision Blizzard, Inc.",Communication Services,Electronic Gaming & Multimedia,59.193844,75.635,6.865368,-0.329,-0.139,35.509388,19.443445,8.045921,0.0062,0.59,0.006194,2022-04-13,0.0,9.432,0.322139,0.031269,0.220202,0.256032,buy,Buy,9,11,7,0,0


In [77]:
# yq.Ticker('AAPL').earnings
# pd.DataFrame(yq.Ticker('AAPL').all_modules)

funds

In [ ]:
yq.Ticker('SCHD').fund_performance

{'SCHD': {'maxAge': 1,
  'performanceOverview': {'asOfDate': '2022-12-14 01:00:00',
   'ytdReturnPct': -0.0095,
   'oneYearTotalReturn': 0.026700001,
   'threeYearTotalReturn': 0.1433},
  'performanceOverviewCat': {'ytdReturnPct': 0.109,
   'fiveYrAvgReturnPct': 0.15359999},
  'trailingReturns': {'asOfDate': '2022-12-12 01:00:00',
   'ytd': -0.0138792,
   'oneMonth': 0.0096464,
   'threeMonth': 0.051581603,
   'oneYear': 0.0182523,
   'threeYear': 0.1406709,
   'fiveYear': 0.13403301,
   'tenYear': 0.13838139,
   'lastBullMkt': 0.0,
   'lastBearMkt': 0.0},
  'trailingReturnsNav': {'ytd': -0.0138792,
   'oneMonth': 0.0096464,
   'threeMonth': 0.051581603,
   'oneYear': 0.0182523,
   'threeYear': 0.1406709,
   'fiveYear': 0.13403301,
   'tenYear': 0.13838139},
  'trailingReturnsCat': {'ytd': 0.109,
   'oneMonth': -0.001,
   'threeMonth': 0.0326,
   'oneYear': 0.17040001,
   'threeYear': 0.105299994,
   'fiveYear': 0.15359999,
   'tenYear': 0.0655,
   'lastBullMkt': 0.0,
   'lastBearMkt':

In [61]:
yq.Ticker('SCHD').fund_profile

{'SCHD': {'maxAge': 1,
  'styleBoxUrl': 'https://s.yimg.com/lq/i/fi/3_0stylelargeeq1.gif',
  'family': 'Schwab ETFs',
  'categoryName': 'Large Value',
  'legalType': 'Exchange Traded Fund',
  'managementInfo': {'managerName': None, 'managerBio': None},
  'feesExpensesInvestment': {'annualReportExpenseRatio': 0.00059999997,
   'annualHoldingsTurnover': 0.14,
   'totalNetAssets': 75870.54,
   'projectionValues': {}},
  'feesExpensesInvestmentCat': {'annualReportExpenseRatio': 0.0034,
   'annualHoldingsTurnover': 34.69,
   'totalNetAssets': 75870.54,
   'projectionValuesCat': {}},
  'brokerages': []}}

In [62]:
yq.Ticker('SCHD').fund_sector_weightings

,SCHD
0,
realestate,0.0000
consumer_cyclical,0.0747
basic_materials,0.0196
consumer_defensive,0.1339
technology,0.1633
communication_services,0.0487
financial_services,0.2036
utilities,0.0037
industrials,0.1640


In [130]:
yq.Ticker('SCHD').fund_top_holdings

symbol                           holdingName  holdingPercent
symbol row                                                             
SCHD   0      MRK                        Merck & Co Inc          0.0457
       1     AMGN                             Amgen Inc          0.0433
       2      IBM  International Business Machines Corp          0.0425
       3     AVGO                          Broadcom Inc          0.0415
       4      PEP                           PepsiCo Inc          0.0411
       5      BLK                         BlackRock Inc          0.0405
       6      LMT                  Lockheed Martin Corp          0.0405
       7       VZ            Verizon Communications Inc          0.0396
       8     CSCO                     Cisco Systems Inc          0.0396
       9      PFE                            Pfizer Inc          0.0396

In [129]:
yq.Ticker('SPHD').fund_top_holdings

symbol                      holdingName  holdingPercent
symbol row                                                        
SPHD   0      IRM                Iron Mountain Inc          0.0321
       1       MO                 Altria Group Inc          0.0314
       2      PPL                         PPL Corp          0.0308
       3      WMB           Williams Companies Inc          0.0291
       4        T                         AT&T Inc          0.0289
       5      KMI        Kinder Morgan Inc Class P          0.0272
       6       PM  Philip Morris International Inc          0.0253
       7      CVX                     Chevron Corp          0.0244
       8      PRU         Prudential Financial Inc          0.0234
       9     AMCR        Amcor PLC Ordinary Shares          0.0232

In [68]:
yq.Ticker('SCHD').price

{'SCHD': {'maxAge': 1,
  'preMarketChangePercent': -0.0105154,
  'preMarketChange': -0.809998,
  'preMarketTime': '2022-12-15 15:29:58',
  'preMarketPrice': 76.22,
  'preMarketSource': 'FREE_REALTIME',
  'postMarketChangePercent': -0.00039754115,
  'postMarketChange': -0.030006409,
  'postMarketTime': 1671143702,
  'postMarketPrice': 75.45,
  'postMarketSource': 'FREE_REALTIME',
  'regularMarketChangePercent': -0.02012197,
  'regularMarketChange': -1.5499954,
  'regularMarketTime': '2022-12-15 22:00:00',
  'priceHint': 2,
  'regularMarketPrice': 75.48,
  'regularMarketDayHigh': 76.4299,
  'regularMarketDayLow': 75.1401,
  'regularMarketVolume': 3815032,
  'regularMarketPreviousClose': 77.03,
  'regularMarketSource': 'DELAYED',
  'regularMarketOpen': 76.34,
  'exchange': 'PCX',
  'exchangeName': 'NYSEArca',
  'exchangeDataDelayedBy': 0,
  'marketState': 'POST',
  'quoteType': 'ETF',
  'symbol': 'SCHD',
  'underlyingSymbol': None,
  'shortName': 'Schwab US Dividend Equity ETF',
  'longNa

In [70]:
yq.Ticker('SCHD').key_stats

{'SCHD': {'maxAge': 1,
  'priceHint': 2,
  'category': 'Large Value',
  'ytdReturn': -0.0095,
  'beta3Year': 0.89,
  'totalAssets': 44960911360,
  'yield': 0.031400003,
  'fundFamily': 'Schwab ETFs',
  'fundInceptionDate': '2011-10-20 02:00:00',
  'legalType': 'Exchange Traded Fund',
  'threeYearAverageReturn': 0.1433,
  'fiveYearAverageReturn': 0.1245,
  'lastSplitFactor': None}}

In [67]:
pd.DataFrame(yq.Ticker('SCHD').all_modules)

,SCHD
assetProfile,"{'phone': 'NA', 'longBusinessSummary': 'To pur..."
defaultKeyStatistics,"{'maxAge': 1, 'priceHint': 2, 'category': 'Lar..."
fundPerformance,"{'maxAge': 1, 'performanceOverview': {'asOfDat..."
fundProfile,"{'maxAge': 1, 'styleBoxUrl': 'https://s.yimg.c..."
pageViews,"{'shortTermTrend': 'DOWN', 'midTermTrend': 'UP..."
price,"{'maxAge': 1, 'preMarketSource': 'FREE_REALTIM..."
quoteType,"{'exchange': 'PCX', 'quoteType': 'ETF', 'symbo..."
summaryDetail,"{'maxAge': 1, 'priceHint': 2, 'previousClose':..."
summaryProfile,"{'phone': 'NA', 'longBusinessSummary': 'To pur..."
topHoldings,"{'maxAge': 1, 'stockPosition': 0.9995, 'bondPo..."


screener

In [244]:
s = yq.Screener()
s.available_screeners
# s.get_screeners('all_cryptocurrencies_us')

['accident_health_insurance',
 'advertising_agencies',
 'aerospace_defense_major_diversified',
 'aerospace_defense_products_services',
 'aggressive_small_caps',
 'agricultural_chemicals',
 'air_delivery_freight_services',
 'air_services_other',
 'all_cryptocurrencies_au',
 'all_cryptocurrencies_ca',
 'all_cryptocurrencies_eu',
 'all_cryptocurrencies_gb',
 'all_cryptocurrencies_in',
 'all_cryptocurrencies_us',
 'aluminum',
 'apparel_stores',
 'appliances',
 'application_software',
 'asset_management',
 'auto_dealerships',
 'auto_manufacturers_major',
 'auto_parts',
 'auto_parts_stores',
 'auto_parts_wholesale',
 'basic_materials',
 'basic_materials_wholesale',
 'beverages_brewers',
 'beverages_soft_drinks',
 'beverages_wineries_distillers',
 'biotechnology',
 'broadcasting_radio',
 'broadcasting_tv',
 'building_materials_wholesale',
 'business_equipment',
 'business_services',
 'business_software_services',
 'catalog_mail_order_houses',
 'catv_systems',
 'cement',
 'chemicals_major_dive